In [ ]:
import fiona
import math

import numpy as np

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cartopy.io.img_tiles as cimgt

from haversine import distance

In [ ]:
from shapely.geometry import shape

class Route:
    def __init__(self, filepath):
        self.filepath = filepath
        self.coordinates = None
        self.bounds = None

    @property
    def coords(self):
        if not self.coordinates:
            self.load_file()
        return self.coordinates

    def shape(self):
        data = {'type': 'MultiLineString', 'coordinates': self.coords}
        return shape(data)

    def load_file(self):
        layer = fiona.open(self.filepath, layer='tracks')
        self.bounds = layer.bounds
        geom = layer[0]
        self.coordinates = geom['geometry']['coordinates']

In [ ]:
class Routes:
    def __init__(self):
        self.routes = []
        self.bounds = [100.0, 100.0, -100.0, -100.0]

    def add(self, route):
        route.load_file()
        self.routes.append(route)
        self.bounds = [
            min(self.bounds[0], route.bounds[0]),
            min(self.bounds[1], route.bounds[1]),
            max(self.bounds[2], route.bounds[2]),
            max(self.bounds[3], route.bounds[3]),
        ]


In [ ]:
routes = Routes()

from glob import glob

for path in glob('routes/*.gpx'):
    routes.add(Route(path))

print(f'{len(routes.routes)} routes loaded')

In [ ]:
def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(12, 17),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
image_factory = cimgt.OSM()  # Open Street Map

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = routes.bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

fig, ax = make_map(projection=image_factory.crs)
ax.set_extent(extent)

# the second argument in that add_image() call is passed to the image_factory and is the "target_z".
# see https://github.com/SciTools/cartopy/blob/master/lib/cartopy/io/img_tiles.py
# OSM() inherits from GoogleWTS, which holds the image_for_domain() method, which is where
# the 14 gets used.
# target_z seems to be more "zoom" than (what I thought at first) "z-order"
# 1-3, 5: no map
# 4: some kind of blocks, quite large
# 6: smaller blocks, noobvious pattern
# 7: similar to 6, some green appearing
# 12 seems to include a lot of place names, 13 has the names much smaller, while 14 makes then tiny
img = ax.add_image(image_factory, 16)  

def random_color():
    return (0.5 * np.random.random_sample() + 0.25,
            0.5 * np.random.random_sample() + 0.25,
            0.5 * np.random.random_sample() + 0.25,)

for route in routes.routes:
    s = ax.add_geometries(route.shape(), ccrs.PlateCarree(),
                        facecolor='none',
                        edgecolor='blue', #(random_color()),
                        linewidth=2)
